# Feature Selection

This notebook is trying out different strategies for feature selection based on sklearn as well as a few neural approaches.

## Dataset

Download dataset to parent data directory.

!wget -P ../data/ https://archive.ics.uci.edu/static/public/169/dorothea.zip

In [ ]:
import pandas as pd

data = []

with open('../data/DOROTHEA/dorothea_train.data', 'r') as f:
    for line in f:
        active_features = line.strip().split()
        data.append(pd.Series({int(feature): 1 for feature in active_features}))

features = pd.concat(data, axis=1).T.fillna(0).sort_index(axis=1)

features.head()


In [ ]:
targets = pd.read_csv('../data/DOROTHEA/dorothea_train.labels', header=None)

In [ ]:
df = pd.concat([features, targets], axis=1)

In [ ]:
df.head()

In [ ]:
df.rename(columns={0: 'target'}, inplace=True)

In [ ]:
df.head()